In [ ]:
from itertools import product

In [ ]:
with open('input.txt') as fl:
    lns = fl.readlines()
grid = [list(ln.strip()) for ln in lns]
n, m = len(grid), len(grid[0])

In [ ]:
# find the start
for i, j in product(range(n), range(m)):
    if grid[i][j] == 'S': break
start_pos = (i, j)

In [ ]:
# the first move is special, need to look in all 4 directions
# and find the two I can move
pos = []
i, j = start_pos
if grid[i-1][j] in {'|','7','F'}: # north
    pos.append((i-1, j))
if grid[i+1][j] in {'|', 'J', 'L'}: # south
    pos.append((i+1, j))
if grid[i][j-1] in {'-','F','L'}: # west
    pos.append((i, j-1))
if grid[i][j+1] in {'-','7','J'}: # west
    pos.append((i, j+1))

# now we can walk the loop until both positions are the same
mvs = 1
prev_pos = [start_pos, start_pos]
paths = [pos[0]],[pos[1]]
while pos[0] != pos[1]:
    for k in range(len(pos)):
        i, j = pos[k]
        v = grid[i][j]
        if v == '|':
            new_pos = (i+1,j) if (i+1,j) != prev_pos[k] else (i-1,j)
        elif v == '-':
            new_pos = (i,j+1) if (i,j+1) != prev_pos[k] else (i,j-1)
        elif v == '7':
            new_pos = (i+1,j) if (i+1,j) != prev_pos[k] else (i,j-1)
        elif v == 'F':
            new_pos = (i+1,j) if (i+1,j) != prev_pos[k] else (i,j+1)
        elif v == 'J':
            new_pos = (i-1,j) if (i-1,j) != prev_pos[k] else (i,j-1)
        elif v == 'L':
            new_pos = (i-1,j) if (i-1,j) != prev_pos[k] else (i,j+1)
        prev_pos[k] = pos[k]
        pos[k] = new_pos
        paths[k].append(new_pos)
    mvs += 1

In [ ]:
mvs

In [ ]:
# part 2 ... fuck me this is gonna suck

In [ ]:
# get the completed loop
loop = paths[0].copy()
loop.extend(paths[1][-2::-1])
loop.insert(0, start_pos)

In [ ]:
# now we pad the grid with empty space placeholders to let the 
# 'slip in between' case happen, all indexes must be multiplied by 2
padded_grid = []
for i in range(len(grid)):
    row = []
    for v, e in zip(grid[i],['*']*len(grid[0])):
        row.append(v); row.append(e)
    padded_grid.append(row)
    padded_grid.append(['*']*len(grid[0]*2))

In [ ]:
# now make a padded loop
padded_loop = [(2*i,2*j) for (i,j) in loop]
filled_loop = []
for ind in range(len(padded_loop)):
    pos, next_pos = padded_loop[ind], padded_loop[(ind+1)%len(padded_loop)]
    diff = next_pos[0]-pos[0], next_pos[1]-pos[1]
    filled_loop.append(pos)
    filled_loop.append((pos[0] + diff[0]//2, pos[1]+diff[1]//2)) 
padded_loop = filled_loop

In [ ]:
# now we need to take all of the non path space and merge into blobs
# until there's nothing left
to_merge = set(product(range(len(padded_grid)), range(len(padded_grid[0]))))
to_merge -= set(padded_loop)
# now merge a blob
blob_list = []
while len(to_merge)>0:
    blob = [to_merge.pop()]
    new_ones = blob.copy()
    while len(new_ones)>0:
        to_try = new_ones.copy()
        new_ones = []
        for pos in to_try:
            for shift in [(1,0), (-1,0), (0, 1), (0, -1)]:
                new_pos = pos[0] + shift[0], pos[1] + shift[1]
                if new_pos[0]>=0 and new_pos[1]>=0 and new_pos in to_merge \
                        and new_pos not in blob and new_pos not in padded_loop:
                    blob.append(new_pos)
                    new_ones.append(new_pos)
                    to_merge.remove(new_pos)
    blob_list.append(blob)

In [ ]:
# now we just need to clean up the blob list ... remove anything that 
# touches the edge then remove all the filler to get the count.
n, m = len(padded_grid), len(padded_grid[0])
contained = []
for blob in blob_list:
    xs, ys = [p[0] for p in blob], [p[1] for p in blob]
    if min(xs)>0 and max(xs)<n and min(ys)>0 and max(ys)<m:
        contained.extend(blob)
contained = [(x//2, y//2) for x,y in contained if x%2==0 and y%2==0]

In [ ]:
len(contained)